In [1]:
import numpy as np
import pandas as pd

In [ ]:
produk = pd.read_csv('Data Produk Kendaraan.csv')
produk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Nama Asuransi   1000 non-null   object 
 1   kategori        1000 non-null   object 
 2   produk_id       1000 non-null   object 
 3   user_id         1000 non-null   object 
 4   Rating          1000 non-null   float64
 5   Premi Asuransi  1000 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 47.0+ KB


In [3]:
produk

,Nama Asuransi,kategori,produk_id,user_id,Rating,Premi Asuransi
0,Insureka Asuransi Mobil,Kendaraan,P0001,U0058,4.4,1054166
1,Insureka Asuransi Mobil All Risk,Kendaraan,P0002,U0195,3.1,1106040
2,Insureka Asuransi Mobil TLO,Kendaraan,P0003,U0165,3.9,1106040
3,Insureka Asuransi Mobil Comprehensive,Kendaraan,P0004,U0167,4.3,1106040
4,Insureka Asuransi Mobil Wuling,Kendaraan,P0005,U0014,3.2,896610
...,...,...,...,...,...,...
995,Asuransi Kendaraan Bermotor Simas,Kendaraan,P0996,U0073,3.2,100000
996,Asuransi Simas Mikro Autobike,Kendaraan,P0997,U0086,4.2,50000
997,Asurasni Simas Mobil Bonus,Kendaraan,P0998,U0154,4.4,100000
998,Tugu Pratama Indonesia Asuransi TLO,Kendaraan,P0999,U0154,4.7,420000


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from sklearn.preprocessing import StandardScaler

In [5]:
# Mapping user dan produk ke index
user_ids = produk['user_id'].unique()
produk_ids = produk['produk_id'].unique()

user2idx = {u: i for i, u in enumerate(user_ids)}
produk2idx = {p: i for i, p in enumerate(produk_ids)}

produk['user_idx'] = produk['user_id'].map(user2idx)
produk['produk_idx'] = produk['produk_id'].map(produk2idx)

# Fitur input
user_indices = produk['user_idx'].values
produk_indices = produk['produk_idx'].values
premi = produk[['Premi Asuransi']].values
ratings = produk['Rating'].values

# Normalisasi premi
from sklearn.preprocessing import StandardScaler
scaler_premi = StandardScaler()
premi_scaled = scaler_premi.fit_transform(premi)

# Info jumlah
num_users = len(user2idx)
num_products = len(produk2idx)
embedding_dim = 32


In [6]:
# Inputs
user_input = Input(shape=(), dtype=tf.int32, name='user_input')
produk_input = Input(shape=(), dtype=tf.int32, name='produk_input')
premi_input = Input(shape=(1,), dtype=tf.float32, name='premi_input')

# Embedding
user_emb = layers.Embedding(num_users, embedding_dim)(user_input)
produk_emb = layers.Embedding(num_products, embedding_dim)(produk_input)

# Flatten
user_vec = layers.Flatten()(user_emb)
produk_vec = layers.Flatten()(produk_emb)

# Concatenate semua fitur
x = layers.Concatenate()([user_vec, produk_vec, premi_input])

# Dense layers
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
output = layers.Dense(1)(x)

# Model
model = Model(inputs=[user_input, produk_input, premi_input], outputs=output)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ produk_input        │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 32)        │      6,368 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 32)        │     32,000 │ produk_input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 32)        │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 32)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ premi_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 65)        │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0],  │
│                     │                   │            │ premi_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,224 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         33 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 44,705 (174.63 KB)

 Trainable params: 44,705 (174.63 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.fit(
    [user_indices, produk_indices, premi_scaled],
    ratings,
    epochs=10,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 15.4086 - val_loss: 13.7531
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 12.3015 - val_loss: 9.8708
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7.9272 - val_loss: 4.9531
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3.1141 - val_loss: 1.2311
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.7876 - val_loss: 0.6726
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4854 - val_loss: 0.8773
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3223 - val_loss: 0.8418
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2057 - val_loss: 0.7995
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1254 - val_loss: 0.8734
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0621 - val_loss: 0.8935


In [8]:
# Ambil produk unik berdasarkan produk_idx
produk_unique = produk.drop_duplicates(subset='produk_idx').set_index('produk_idx')

In [9]:
def recommend_hybrid_by_user_premi(user_id, premi_user, top_n=10):
    # Scaling premi input user
    premi_user_scaled = scaler_premi.transform([[premi_user]])[0][0]

    # Jika user lama
    if user_id in user2idx:
        user_idx = user2idx[user_id]
        rated_produk_idx = produk[produk['user_id'] == user_id]['produk_idx'].unique()
        unrated_produk_idx = np.setdiff1d(np.arange(len(produk2idx)), rated_produk_idx)
    else:
        print(f"User {user_id} tidak ditemukan, gunakan cold-start berbasis premi user.")
        user_idx = 0  # dummy
        unrated_produk_idx = np.arange(len(produk2idx))
    
    user_array = np.array([user_idx] * len(unrated_produk_idx))
    produk_array = np.array(unrated_produk_idx)
    premi_array = np.array([premi_user_scaled] * len(unrated_produk_idx))

    # Prediksi rating
    pred_ratings = model.predict([user_array, produk_array, premi_array], verbose=0).flatten()

    # Gabungkan hasil prediksi
    rekom_df = pd.DataFrame({
        'produk_idx': produk_array,
        'Rating': pred_ratings  # <- rename ke Rating untuk tampilan
    })
    rekom_df = rekom_df.join(produk_unique[['Nama Asuransi', 'Premi Asuransi']], on='produk_idx')

    # Ambil hanya 3 kolom yang diminta
    return rekom_df.sort_values('Rating', ascending=False).head(top_n)[
        ['Nama Asuransi', 'Premi Asuransi', 'Rating']
    ].reset_index(drop=True)

In [10]:
recommend_hybrid_by_user_premi('U0012', premi_user=600000)

,Nama Asuransi,Premi Asuransi,Rating
0,Insureka Asuransi Mobil TLO,1106040,5.240193
1,Tugu Pratama Indonesia Asuransi All risk,2100000,5.217009
2,Insureka Asuransi Mobil BYD,1030320,5.211643
3,ACA Otomate Smart,306000,5.208796
4,Insureka Asuransi Mobil TLO,1106040,5.194698
5,Insureka Asuransi Mobil All Risk,1106040,5.188691
6,Insureka Asuransi Mobil All Risk,1106040,5.179155
7,Tugu Pratama Indonesia Asuransi All risk,2100000,5.130768
8,Insureka Asuransi Mobil All Risk,1106040,5.111757
9,Insureka Asuransi Mobil BYD,1030320,5.106969


In [11]:
# user baru
recommend_hybrid_by_user_premi('user_baru_123', premi_user=750000)

User user_baru_123 tidak ditemukan, gunakan cold-start berbasis premi user.


,Nama Asuransi,Premi Asuransi,Rating
0,Insureka Asuransi Mobil TLO,1106040,5.257568
1,Tugu Pratama Indonesia Asuransi All risk,2100000,5.234632
2,Insureka Asuransi Mobil BYD,1030320,5.227678
3,ACA Otomate Smart,306000,5.220683
4,Insureka Asuransi Mobil TLO,1106040,5.207483
5,Insureka Asuransi Mobil All Risk,1106040,5.204659
6,Insureka Asuransi Mobil All Risk,1106040,5.197109
7,Tugu Pratama Indonesia Asuransi All risk,2100000,5.142252
8,Insureka Asuransi Mobil All Risk,1106040,5.128238
9,Insureka Asuransi Mobil BYD,1030320,5.122004


In [12]:
model.save('rekomendasi_kendaraan.h5')

In [13]:
import joblib

# Simpan scaler fitur
joblib.dump(scaler_premi , "scaler_rekomendasi_kendaraan.pkl")

['scaler_rekomendasi_kendaraan.pkl']

# INFERENSI

In [16]:
from rekomendasi_kendaraan import recommend_hybrid_by_user_premi

user_id = 'U9999'
premi_input = 750000

rekomendasi = recommend_hybrid_by_user_premi(user_id, premi_input, top_n=10)
rekomendasi

,Nama Asuransi,Premi Asuransi,Rating
0,Insureka Asuransi Mobil TLO,1106040,5.257568
1,Tugu Pratama Indonesia Asuransi All risk,2100000,5.234632
2,Insureka Asuransi Mobil BYD,1030320,5.227678
3,ACA Otomate Smart,306000,5.220683
4,Insureka Asuransi Mobil TLO,1106040,5.207483
5,Insureka Asuransi Mobil All Risk,1106040,5.204659
6,Insureka Asuransi Mobil All Risk,1106040,5.197109
7,Tugu Pratama Indonesia Asuransi All risk,2100000,5.142252
8,Insureka Asuransi Mobil All Risk,1106040,5.128238
9,Insureka Asuransi Mobil BYD,1030320,5.122004
